Para criação dessa automatização é necessário que exista uma "lista_tratados.txt", no mesmo endereço que deste, que catalogue os meses em que já foram processados.\
Este pipeline então requisita a lista de datasets existentes no database e então compara com o catálogo, se existir datasets que não estão catalogados o processo de tratamento é executado, senão nada acontece.

# Carregando bibliotecas

In [2]:
import grequests
import pandas as pd
import re
import requests
import warnings
import urllib.parse
import datetime

warnings.simplefilter("ignore", category=UserWarning)

# Definindo Variáveis para usar o API

In [3]:
#Url para chamar a API e escolha dos métodos

base_endpoint = 'https://opendata.nhsbsa.net/api/3/action/'
package_list_method = 'package_list'     # Lista dos datasets no portal
package_show_method = 'package_show?id=' # Lista todos recursos de dataset
action_method = 'datastore_search_sql?'  # método para ação SQL

# Automatização

Como o volume de dados é muito grande, assumiu-se que as consultas e resultados desejados nesta automação para os consecutivos meses são as mesmas que foram feitas anteriormente, e não todos os dados.

In [4]:
#Obtendo lista de Datasets.
datasets_response = requests.get(base_endpoint + "package_show?id=english-prescribing-data-epd").json()

#listando todos os metadados dos datasets.
lista_df  = pd.json_normalize(datasets_response['result']['resources'])

#selecionando apenas os nomes dos datasets.
listameses = lista_df['name']

#Transformando lista em formato panda series para python list.
listameses = listameses.tolist()

In [5]:
#Importando a lista de dados já tratados.
arquivo = open('lista_tratados.txt', 'r')

#criando lista com todos datasets já tratados.
listatratados = [i for i in arquivo.readlines()]

#retirando \n
listatratados = [i.replace('\n', '') for i in listatratados]


In [6]:
#lista que terá os dados não tratados, começa como uma cópia da lista de datasets no site.
listantratados = listameses.copy()

#removendo a lista de dados tratados da lista dos datasets no site.
for i in listatratados:
    if i in listantratados:
        listantratados.remove(i)

In [8]:
if listantratados != []:
    for j in listantratados:
        
        #Dataset para o mês
        nome_dataset = j

        #criando variáveis com as datas
        mes = nome_dataset[len(nome_dataset)-2:len(nome_dataset)]
        ano = nome_dataset[len(nome_dataset)-6:len(nome_dataset)-2]

        #-----------------------------------------------------------------------------
        #Regiões químicos mais vendidos por região
        #-----------------------------------------------------------------------------

        #Criando o query para obter todas regiões no dataset
        query_todas_regioes = "SELECT " \
                            "DISTINCT regional_office_name " \
                            f"FROM `{nome_dataset}` "

        #Método para codificar o método em uma url

        todas_regioes_api_call = f"{base_endpoint}" \
                                f"{action_method}" \
                                "resource_id=" \
                                f"{nome_dataset}" \
                                "&" \
                                "sql=" \
                                f"{urllib.parse.quote(query_todas_regioes)}"

        #Resposta JSON
        json_regioes = requests.get(todas_regioes_api_call).json()

        #Extrai os dados requisitados para um dataframe
        todas_regioes_df  = pd.json_normalize(json_regioes['result']['result']['records'])

        regioes_df = pd.DataFrame()

        #for loop para extrair cada dataset regional e então concatená-los
        for i in range(0, len(todas_regioes_df)) :
        
            #Query para uma região
            query_uma_regiao = "SELECT " \
                                    "regional_office_name, " \
                                    "chemical_substance_bnf_descr, " \
                                    "sum(total_quantity) as quantity " \
                                    "FROM " \
                                    f"`{nome_dataset}` " \
                                    "WHERE " \
                                    "regional_office_name = " f"'{todas_regioes_df.regional_office_name[i]}' " \
                                    "GROUP BY " \
                                    "regional_office_name, " \
                                    "chemical_substance_bnf_descr " \
                                    "ORDER BY " \
                                    "quantity DESC " \
                                    "LIMIT 15"

            #criação da url de requisição ao API
            uma_regiao_api_call = f"{base_endpoint}" \
                                    f"{action_method}" \
                                    "resource_id=" \
                                    f"{nome_dataset}" \
                                    "&" \
                                    "sql=" \
                                    f"{urllib.parse.quote(query_uma_regiao)}"

            #Resultado em JSON temporário
            json_uma_regiao = requests.get(uma_regiao_api_call).json()

            #Extração apenas dos resultados do JSON e transformação em um dataframe
            uma_regiao_df  = pd.json_normalize(json_uma_regiao['result']['result']['records'])

            #Concatenação dos dataframes de região
            frames = [regioes_df, uma_regiao_df]
            regioes_df = pd.concat(frames, sort = False)

        #Arrumando o índice
        regioes_df = regioes_df.reset_index(drop = True)

        #Exportando para um CSV
        regioes_df.to_csv(f'automatizados/regiaotop10_{ano}{mes}.csv')

        #-----------------------------------------------------------------------------
        #10 Químicos mais custosos por mês
        #-----------------------------------------------------------------------------

        #Criação do dataset em que será concatenado
        custosos_df = pd.DataFrame()

        #Query para um mês
        query_um_mes = "SELECT " \
                                "chemical_substance_bnf_descr, " \
                                "sum(actual_cost) as cost " \
                                "FROM " \
                                f"`{nome_dataset}` " \
                                "GROUP BY " \
                                "chemical_substance_bnf_descr " \
                                "ORDER BY " \
                                "cost DESC " \
                                "LIMIT 10"

        #criação da url de requisição ao API
        um_mes_api_call = f"{base_endpoint}" \
                                f"{action_method}" \
                                "resource_id=" \
                                f"{nome_dataset}" \
                                "&" \
                                "sql=" \
                                f"{urllib.parse.quote(query_um_mes)}"

        #Resultado em JSON temporário
        json_um_mes = requests.get(um_mes_api_call).json()

        #Extração apenas dos resultados do JSON e transformação em um dataframe
        um_mes_df  = pd.json_normalize(json_um_mes['result']['result']['records'])

        #Concatenação dos dataframes mensais
        frames = [custosos_df, um_mes_df]
        custosos_df = pd.concat(frames, sort = False)

        #Agrupando e ordenando os 10 químicos mais custosos em todos os meses
        custosos_df = custosos_df.groupby('chemical_substance_bnf_descr').sum().sort_values('cost', ascending=False).head(10)

        #Arrumando o índice
        custosos_df = custosos_df.reset_index(drop = False)

        #Visualização
        custosos_df
        
        #exportação para CSV dos químicos mais custosos em todos os meses
        custosos_df.to_csv(f'automatizados/custotop10_{ano}{mes}.csv')   

        #-----------------------------------------------------------------------------
        #Químicos mais prescritos por mês
        #-----------------------------------------------------------------------------

        #Criação do dataset em que será concatenado
        prescritos_df = pd.DataFrame()

        #Query para um mês
        query_um_mes = "SELECT " \
                                "bnf_description, " \
                                "items " \
                                "FROM " \
                                f"`{nome_dataset}` " \
                                "LIMIT 10"

        #criação da url de requisição ao API
        um_mes_api_call = f"{base_endpoint}" \
                                f"{action_method}" \
                                "resource_id=" \
                                f"{nome_dataset}" \
                                "&" \
                                "sql=" \
                                f"{urllib.parse.quote(query_um_mes)}"

        #Resultado em JSON temporário
        json_um_mes = requests.get(um_mes_api_call).json()

        #Extração apenas dos resultados do JSON e transformação em um dataframe
        um_mes_df  = pd.json_normalize(json_um_mes['result']['result']['records'])

        #Concatenação dos dataframes mensais
        frames = [prescritos_df, um_mes_df]
        prescritos_df = pd.concat(frames, sort = False)

        #Agrupando e ordenando os 10 medicamentos mais prescritos em todos os meses
        prescritos_df = prescritos_df.groupby('bnf_description').sum().sort_values('items', ascending=False).head(10)

        #Arrumando o índice
        prescritos_df = prescritos_df.reset_index(drop = False)

        #exportando o CSV
        prescritos_df.to_csv(f'automatizados/prescritos_{ano}{mes}.csv')

        #-----------------------------------------------------------------------------
        #Mais prescritos por prescriber
        #-----------------------------------------------------------------------------

        #lista com todas as prescribers
        prescribers_df = pd.DataFrame()

        #Query para um mês
        query_um_mes = "SELECT " \
                    "T.practice_name, " \
                    "T.chemical_substance_bnf_descr, " \
                    "T.items " \
                    "FROM " \
                    f"`{nome_dataset}` as T " \
                    "WHERE " \
                    f"T.items = ( SELECT MAX(items) FROM `{nome_dataset}` WHERE practice_name = T.practice_name)" 

        #criação da url de requisição ao API
        um_mes_api_call = f"{base_endpoint}" \
                            f"{action_method}" \
                            "resource_id=" \
                            f"{nome_dataset}" \
                            "&" \
                            "sql=" \
                            f"{urllib.parse.quote(query_um_mes)}"

        #Resultado em JSON temporário
        json_um_mes = requests.get(um_mes_api_call).json()

        #Extração apenas dos resultados do JSON e transformação em um dataframe
        um_mes_df  = pd.json_normalize(json_um_mes['result']['result']['records'])

        #dataframes de prescribers
        frames = [prescribers_df, um_mes_df]
        prescribers_df = pd.concat(frames, sort= False)

        #limpeza e tratamento dos dados concatenados
        prescribers_df = prescribers_df.groupby(['practice_name','chemical_substance_bnf_descr'])['items'].sum().reset_index().sort_values(
            ['practice_name', 'items'], ascending=False).drop_duplicates(
            ['practice_name']).sort_values('practice_name')

        #arrumando o índice
        prescribers_df = prescribers_df.reset_index(drop = True)

        #Exportando para CSV
        prescribers_df.to_csv(f'automatizados/prescribers_{ano}{mes}.csv')

        #-----------------------------------------------------------------------------
        #Prescribers adicionadas
        #-----------------------------------------------------------------------------

        #criando a variável que armazenará o mês anterior ao dataset
        if int(mes) == 1:
            mes0 = 12
            ano = int(nome_dataset[len(nome_dataset)-6:len(nome_dataset)-3])-1
            nome_dataset0 = f'EPD_{ano}{mes0}'
        else:
            mes0 = int(mes) - 1
            if mes0 < 10:
                mes0 = f"0{mes0}"
            nome_dataset0 = f'EPD_{ano}{mes0}'

        #Query para o mês 6 para quantidade de prescribers
        query_mes0 = "SELECT " \
                        "count(distinct(practice_name))" \
                        "FROM " \
                        f"`{nome_dataset0}`" \

        #criação da url de requisição ao API
        mes0_api_call = f"{base_endpoint}" \
                                f"{action_method}" \
                                "resource_id=" \
                                f"{nome_dataset0}" \
                                "&" \
                                "sql=" \
                                f"{urllib.parse.quote(query_mes0)}"

        #Resultado em JSON
        json_mes0 = requests.get(mes0_api_call).json()

        #Extração apenas dos resultados do JSON e transformação em um dataframe
        mes0_df  = pd.json_normalize(json_mes0['result']['result']['records'])

        #Query para o mês 7 para quantidade de prescribers
        query_mes1 = "SELECT " \
                        "count(distinct(practice_name))" \
                        "FROM " \
                        f"`{nome_dataset}`" \

        #criação da url de requisição ao API
        mes1_api_call = f"{base_endpoint}" \
                                f"{action_method}" \
                                "resource_id=" \
                                f"{nome_dataset}" \
                                "&" \
                                "sql=" \
                                f"{urllib.parse.quote(query_mes1)}"

        #Resultado em JSON
        json_mes1 = requests.get(mes1_api_call).json()

        #Extração apenas dos resultados do JSON e transformação em um dataframe
        mes1_df  = pd.json_normalize(json_mes1['result']['result']['records'])

        #Diferença é o acréscimo de prescritores.
        diff = mes1_df.f0_[0] - mes0_df.f0_[0]

        #Exportando a diferença para um txt
        with open(f'automatizados/aumentoPrescriber_{ano}{mes}.txt', 'w') as f:
            if diff > 0:
                f.write(f'Houve um acréscimo de {diff} prescribers no mês {mes} em relação ao {mes0}, no ano de {ano}')
            elif diff < 0:
                diff = diff * -1
                f.write(f'Houve uma diminuição de {diff} prescribers no mês {mes} em relação ao {mes0}, no ano de {ano}')
            else:
                f.write(f'Não houve alteração no número de prescribers entre {mes0} e {mes}')
        #-----------------------------------------------------------------------------
        #Prescribers em mais de uma região
        #-----------------------------------------------------------------------------

        #Criação do dataset em que será concatenado
        presregiao_df = pd.DataFrame()

        #Query para um mês
        query_um_mes = "SELECT " \
                                "practice_name, " \
                                "COUNT(DISTINCT(regional_office_name)) as regqt " \
                                "FROM " \
                                f"`{nome_dataset}` " \
                                "GROUP BY " \
                                    "practice_name"

        #criação da url de requisição ao API
        um_mes_api_call = f"{base_endpoint}" \
                                f"{action_method}" \
                                "resource_id=" \
                                f"{nome_dataset}" \
                                "&" \
                                "sql=" \
                                f"{urllib.parse.quote(query_um_mes)}"

        #Resultado em JSON temporário
        json_um_mes = requests.get(um_mes_api_call).json()

        #Extração apenas dos resultados do JSON e transformação em um dataframe
        um_mes_df  = pd.json_normalize(json_um_mes['result']['result']['records'])

        #Concatenação dos dataframes mensais
        frames = [presregiao_df, um_mes_df]
        presregiao_df = pd.concat(frames, sort = False)

        #arrumando o dataframe para facilitar a leitura
        presregiao_df = presregiao_df[['practice_name', 'regqt']]

        #excluindo linhas em que as regiões são iguais a 1
        presregiao_df = presregiao_df[presregiao_df.regqt > 1]

        #Ordenando os valores
        presregiao_df = presregiao_df.sort_values('regqt', ascending=False)

        #Arrumando o índice
        presregiao_df = presregiao_df.reset_index(drop = True)
        
        #Exportando para CSV
        presregiao_df.to_csv(f'automatizados/precriberPorRegiao_{ano}{mes}.csv')

        #-----------------------------------------------------------------------------
        #Preço médio dos prescritos
        #-----------------------------------------------------------------------------

        #Criação do dataset em que será concatenado
        medio_df = pd.DataFrame()

        #Query para um mês
        query_um_mes = "SELECT " \
                                "distinct(bnf_description), " \
                                "sum(total_quantity) as qt , " \
                                "sum(actual_cost) as cost " \
                            "FROM " \
                                f"`{nome_dataset}` " \
                            "GROUP BY " \
                                "bnf_description"

        #criação da url de requisição ao API
        um_mes_api_call = f"{base_endpoint}" \
                                f"{action_method}" \
                                "resource_id=" \
                                f"{nome_dataset}" \
                                "&" \
                                "sql=" \
                                f"{urllib.parse.quote(query_um_mes)}"

        #Resultado em JSON temporário
        json_um_mes = requests.get(um_mes_api_call).json()

        #Extração apenas dos resultados do JSON e transformação em um dataframe
        medio_df  = pd.json_normalize(json_um_mes['result']['result']['records'])

        #Criando a coluna de média
        medio_df['mean'] = medio_df.cost/medio_df.qt 

        #excluindo colunas de custo e quantidade
        medio_df.drop(['cost','qt'], axis = 1, inplace= True)

        #Arrumando o índice
        medio_df = medio_df.reset_index(drop = True)

        #exportando CSV
        medio_df.to_csv(f'automatizados/medio_{ano}{mes}.csv')

        #-----------------------------------------------------------------------------
        #Preço prescrição de maior valor
        #-----------------------------------------------------------------------------

        #dataframe que receberá os dados
        maiorvalor_df = pd.DataFrame()

        #Query para um mês
        query_um_mes = "SELECT " \
                        "T.practice_name, " \
                        "T.bnf_description, " \
                        "T.actual_cost " \
                        "FROM " \
                        f"`{nome_dataset}` as T " \
                        "WHERE " \
                        f"T.actual_cost = ( SELECT MAX(actual_cost) FROM `{nome_dataset}` WHERE practice_name = T.practice_name)" 

        #criação da url de requisição ao API
        um_mes_api_call = f"{base_endpoint}" \
                                f"{action_method}" \
                                "resource_id=" \
                                f"{nome_dataset}" \
                                "&" \
                                "sql=" \
                                f"{urllib.parse.quote(query_um_mes)}"

        #Resultado em JSON temporário
        json_um_mes = requests.get(um_mes_api_call).json()

        #Extração apenas dos resultados do JSON e transformação em um dataframe
        um_mes_df  = pd.json_normalize(json_um_mes['result']['result']['records'])

        #dataframes de prescribers
        frames = [maiorvalor_df, um_mes_df]
        maiorvalor_df = pd.concat(frames, sort= False)

        #Arrumando o índice
        maiorvalor_df = maiorvalor_df.reset_index(drop = True)

        #Exportando para CSV
        maiorvalor_df.to_csv(f'automatizados/maiorValorPorPrescriber_{ano}{mes}.csv')

        #-----------------------------------------------------------------------------
        #-----------------------------------------------------------------------------
        #Escrevendo na lista de verificação o mês tratado

        #abrindo o arquivo de lista
        with open('lista_tratados.txt', 'a') as arquivo:
            arquivo.write(f'{j}\n')